#### reference code source : https://medium.com/@sarves021999/air-signature-creation-using-opencv-40f1536ecb8b
#### if mediapipe getting install error then use collab
##### collab link : https://colab.research.google.com/drive/1ZQpaf0qaylLKhRH1EdS80sPDgUcu1Lo_#scrollTo=zedIZIYmwjzl

In [54]:
import cv2
import mediapipe as np
import time
import math

class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.npHands = np.solutions.hands
        self.hands = self.npHands.Hands(self.mode, self.maxHands, self.detectionCon, self.trackCon)
        self.npDraw = np.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]

In [55]:
def findHands(self, img, draw=True):
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    self.results = self.hands.process(imgRGB)

    if self.results.multi_hand_landmarks:
        for handLMS in self.results.multi_hand_landmarks:
            if draw:
                self.npDraw.draw_landmarks(img, handLMS, self.npHands.HAND_CONNECTIONS)
    return img

In [56]:
def findPosition(self, img, handNo=0, draw=True):
    self.lmlist = []

    if self.results.multi_hand_landmarks:
        myhand = self.results.multi_hand_landmarks[handNo]
        for id, lm in enumerate(myhand.landmark):
            h, w, c = img.shape
            cx, cy = int(lm.x * w), int(lm.y * h)
            self.lmlist.append([id, cx, cy])

            if draw:
                cv2.circle(img, (cx, cy), 7, (255, 0, 255), cv2.FILLED)

    return self.lmlist

In [57]:
def fingerUp(self):
    finger = []
    # Thumb
    if self.lmlist[self.tipIds[0]][1] < self.lmlist[self.tipIds[0] - 1][1]:
        finger.append(0)
    else:
        finger.append(1)

    # 4 fingers
    for id in range(1, 5):
        if self.lmlist[self.tipIds[id]][2] < self.lmlist[self.tipIds[id] - 2][2]:
            finger.append(1)
        else:
            finger.append(0)
    return finger

In [58]:
# !pip install HandTrackingModule
# !pip install cvzone


In [59]:
# Import opencv for computer vision stuff
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image
# Import hand Tracking Module
from cvzone.HandTrackingModule import HandDetector

class AirSigning:
    def __init__(self, defaultCam=0):
        self.primaryCam = defaultCam
        self.camHeight, self.camWidth = 540, 960
        self.detector = HandDetector(maxHands=1)
        self.drawColor = (141, 43, 193)
        self.brushThickness = 5
        self.smooth = 3
        self.retrySign = 0

    def drawSign(self):
        # Connect to webcam
        cap = cv2.VideoCapture(self.primaryCam)
        self.camWidth = int(cap.get(3))
        self.camHeight = int(cap.get(4))

        # Sign Area rectangle
        rectIniWid, rectIniHei = int(self.camWidth * 0.1), int(self.camHeight * 0.1)
        rectEndWid, rectEndHei = int(self.camWidth * 0.9), int(self.camHeight * 0.4)
        xPrevious, yPrevious = 0, 0
        imgCanvas = np.zeros((self.camHeight, self.camWidth, 3), np.uint8)

        while cap.isOpened():
            ret, frame = cap.read()

            frame = cv2.flip(frame, 1)
            frame = self.detector.findHands(frame)
            lmList = self.detector.findPosition(frame, draw=False)

            if len(lmList) != 0:
                # tip of index finger
                indFx, indFy = lmList[8][1:]

                # fingers up detection
                fingers = self.detector.fingerUp()

                # Pause Mode
                if fingers[1] and fingers[2]:
                    xPrevious, yPrevious = 0, 0

                # Draw Mode
                if fingers[1] and not fingers[2] and not fingers[3] and not fingers[4] and \
                        rectIniWid < indFx < rectEndWid and rectIniHei < indFy < rectEndHei:

                    cv2.circle(frame, (indFx, indFy), 10, self.drawColor, cv2.FILLED)

                    if xPrevious == 0 and yPrevious == 0:
                        xPrevious, yPrevious = indFx, indFy

                    indFx = xPrevious + (indFx - xPrevious) // self.smooth
                    indFy = yPrevious + (indFy - yPrevious) // self.smooth

                    cv2.line(imgCanvas, (xPrevious, yPrevious), (indFx, indFy), self.drawColor, self.brushThickness)
                    xPrevious, yPrevious = indFx, indFy

                if fingers[1:4] == [1, 1, 1]:
                    imgCanvas = np.zeros((self.camHeight, self.camWidth, 3), np.uint8)

            imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)
            _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)
            imgInv = cv2.cvtColor(imgInv, cv2.COLOR_GRAY2BGR)
            frame = cv2.bitwise_and(frame, imgInv)
            frame = cv2.bitwise_or(frame, imgCanvas)

            frame = cv2.rectangle(frame, (rectIniWid, rectIniHei), (rectEndWid, rectEndHei), (0, 78, 0), 2)

            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.8
            position = (self.camWidth // 2, self.camHeight - 30)
            color = (0, 0, 0)
            cv2.putText(frame, "After draw press 'Q' to continue - >", position, font, font_scale, color, 2, cv2.LINE_AA)

            cv2.imshow('Webcam', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                pngOfSign = self.removeBlackBackground(imgCanvas)
                cv2.imwrite("\\tempSign.png",
                            pngOfSign[rectIniHei:rectEndHei, rectIniWid:rectEndWid])
                break

        cap.release()
        cv2.destroyAllWindows()

        self.retrySign += 1

        return "\\tempSign.png"



In [60]:
def removeBlackBackground(self, imgCanvas):
    # convert the image to grayscale
    gray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)

    # threshold the image to create a mask
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)

    # invert the mask
    mask_inv = cv2.bitwise_not(mask)

    # apply the mask to the image
    img_masked = cv2.bitwise_and(imgCanvas, imgCanvas, mask=mask)

    # add an alpha channel to the image
    alpha = np.ones(imgCanvas.shape[:2], dtype=np.uint8) * 255
    alpha[mask_inv == 255] = 0

    return cv2.merge((img_masked, alpha))

In [61]:
if __name__ == "__main__":
    signComponent = AirSigning()
    signComponent.drawSign()